In [ ]:
%matplotlib inline
from IPython.display import clear_output
import numpy as np
from scipy.stats import multivariate_normal as mv_norm
import matplotlib.pyplot as plt
import time
plt.figure(figsize=(20, 40))

In [ ]:
class gausianDataGenerator:
    def __init__(self, mean, var, size):
        self.mean = mean
        self.var = var
        self.size = size
        self.U = []
        self.V = []
        self.X = []
    
    def _generateUniformData(self):
        self.U = np.random.uniform(0, 1, self.size)
        self.V = np.random.uniform(0, 1, self.size)
    
    def generateGausianData(self):
        self._generateUniformData()
        self.X = np.sqrt(self.var) * (np.sqrt(-2 * np.log(self.U)) * np.cos(2 * np.pi * self.V)) + self.mean
    #    self.Y = self.var * (np.sqrt(-2 * np.log(self.U)) * np.sin(2 * np.pi * self.V)) + self.mean
        return self.X
    
    def drawFigure(self):
        plt.title('U_data')
        plt.hist(self.U, alpha = 0.75, bins = 50, facecolor='g')
        plt.show()
        plt.title('V_data')
        plt.hist(self.V, alpha = 0.75, bins = 50, facecolor='g')
        plt.show()
        plt.title('X_data')
        plt.hist(self.X, alpha = 0.75, bins = 50, facecolor='g')
        plt.show()

In [ ]:
gdg = gausianDataGenerator(5, 3, 1000)
e = gdg.generateGausianData()
print('Real mean:', np.mean(e))
print('Real variance:', np.var(e))
gdg.drawFigure()

In [ ]:
def sequentialEstimate(m, s, size):
    currentMean = 0
    currentVar = 0
    M = 0
    gen =  gausianDataGenerator(m, s, 1)
    for idx in range(1, size + 1):
        points = gen.generateGausianData()
        point = points[0]
        tmp = point - currentMean
        currentMean = (currentMean + point) / idx 
        M += tmp * (point - currentMean)
        currentVar = M / (idx)
        print('Iteration:', idx)
        print('New point is:', round(point, 4))
        print('Current Mean/answer:', round(currentMean, 4), '/', m)
        print('Current variance/answer:', round(currentVar, 4), '/', s, '\n')

sequentialEstimate(0, 5, 500)
    

In [ ]:
class polyLinearDataGenerator:
    def __init__(self, n, a, w, size):
        self.bases = n
        self.var = a
        self.W = w
        self.X = []
        self.Y = []
        self.A = []
        self.line = []
        self.size = size
        
    def _generateUniformData(self):
        self.X = np.random.uniform(-10, 10, self.size)
    
    def _generateBasisMatrix(self):
        self.A = np.zeros(shape=[self.size, self.bases])
        for r in range(self.size):
            x = 1
            for c in range(self.bases):
                self.A[r][self.bases-c-1] = x
                x = x * self.X[r]
        
    def generateLinearData(self):
        self._generateUniformData()
        self._generateBasisMatrix()
        self.line = np.matmul(self.A, self.W)
        
        g = gausianDataGenerator(0, self.var, len(self.X))
        bias = g.generateGausianData().reshape([len(self.X), 1])
        self.Y = self.line + bias
        
        return self.X, self.Y
    
    def drawFigure(self):
        plt.title('Data with line generator')
        plt.plot(self.X, self.line, color='green', zorder=10)
        plt.scatter(self.X, self.Y, label='point', marker='o', edgecolors='blue', facecolors='none')

In [ ]:
w = np.array([[1], [1]])
ldg = polyLinearDataGenerator(len(w), 5, w , 100)
Y = ldg.generateLinearData()
ldg.drawFigure()

In [ ]:
class bayesianLinearModel:
    def __init__(self, basis, a, b, w, n, density, stdev):
        """
        self.S -> new prior's cov inverse
        self.M -> new prior's mean
        self.size -> data point number
        self.basis -> bases number
        """
        self.W = w
        self.a = a
        self.precision = b
        self.basis = basis
        self.S = np.eye(basis) * (1 / b)
        self.M = np.zeros([basis, 1])
        self.size = n
        self.posterior = mv_norm(mean = np.reshape(self.M, basis), cov= self.S)
        self.density = density
        self.stdev = stdev
     
  #  def _mutivariateGaussian(self):
        
    def _makeDesignMatrix(self, x):
        n = len(x)
        X = np.empty([n, self.basis])
        
        for i in range(n):
            t = 1 #temp
            for j in range(self.basis):
                X[i][self.basis-j-1] = t
                t = t * x[i]
            
        return X
        
    def _drawResult(self, dataX, dataY, x):
        X = self._makeDesignMatrix(x)
        target = np.matmul(X, self.W)      
        y_mean = np.matmul(X, self.M)
        y_cov = (1 / self.a) + np.matmul(np.matmul(X, np.linalg.inv(self.S)), np.transpose(X))
        
        print(y_mean, y_cov, '\n')

        std = np.sqrt(np.diag(y_cov))
        y_upper = y_mean.flatten() + self.stdev * std
        y_lower = y_mean.flatten() - self.stdev * std
        y_intervel = np.empty(len(x))
        for i in range(len(x)):
            if i % 2:
                y_intervel[i] = y_upper[i]
            else:
                y_intervel[i] = y_lower[i]
        
      #  clear_output(wait=True)
      #  plt.subplot(1, 2, 1)
        '''
        plt.ylim(np.min(target), np.max(target))
        plt.plot(x, target, color = 'green')
        plt.plot(x, y_mean, color = 'red')
        plt.plot(x, y_intervel, c='pink', linewidth = 10, zorder=-1)
        plt.scatter(dataX, dataY, marker = 'o', facecolors='none', edgecolors='b')
        plt.draw()
        plt.pause(0.0001)
        '''

     #   plt.subplot(1, 2, 2)
    '''
        prior_x, prior_y = np.mgrid[-5:5:.01, -5:5:.01]
        pos = np.empty(prior_x.shape + (2,))
        pos[:, :, 0] = prior_x
        pos[:, :, 1] = prior_y
        plt.contourf(prior_x, prior_y, self.posterior.pdf(pos), 20, cmap='rainbow')
        plt.xlabel('$w_0$', fontsize=16)
        plt.ylabel('$w_1$', fontsize=16)
        plt.scatter(self.W[0][0], self.W[1][0], marker='+', c='black', s=60)
        plt.draw()
        plt.pause(0.1)
'''''
    def onlineLearning(self):
        dataGenerator = polyLinearDataGenerator(self.basis, self.a, self.W, 1)
        predictiveData = np.linspace(-10, 10, self.density)
        dataX = []
        dataY = []
        self._drawResult(dataX, dataY, predictiveData)
        time.sleep(1)
        
        for _ in range(self.size):
            x, y = dataGenerator.generateLinearData()
            dataX.append(x)
            dataY.append(y)
            X = self._makeDesignMatrix(x)
            np.reshape(y, (1, 1))
            
            m = self.M
            s = self.S   
            
            rightMatrix = self.a * np.matmul(np.transpose(X), y) + np.matmul(s, m)
            self.S = self.a * np.matmul(np.transpose(X), X) + s
            self.M = np.matmul(np.linalg.inv(self.S), rightMatrix)
            self._drawResult(dataX, dataY, predictiveData)
            time.sleep(0.1)
            self.posterior = mv_norm(mean = np.reshape(self.M, self.basis), cov= np.linalg.inv(self.S))
    

In [ ]:
model = bayesianLinearModel(len(w), 0.5, 100, w, 20, density = 100, stdev = 5)
model.onlineLearning()